In [1]:
# import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
# preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# import datasets 
train_labels = pd.read_csv("datasets/train_labels.csv", delimiter=",", header=0, index_col=0)
train_images = np.load("datasets/train_images.npy", encoding="latin1")
test_images = np.load("datasets/train_images.npy", encoding="latin1")

In [3]:
# store copies, may wish to retrieve original datasets
copy_train_labels = np.copy(train_labels)
copy_train_images = np.copy(train_images)
copy_test_images = np.copy(test_images)
 
# restore original datasets
def restore():
    train_labels = np.copy(copy_train_labels)
    train_images = np.copy(copy_train_images)
    test_images = np.copy(copy_test_images)
    return train_labels, train_images, test_images

In [4]:
# check shape
print(train_labels.shape)
print(train_images.shape)
print(test_images.shape)

(10000, 1)
(10000, 2)
(10000, 2)


In [62]:
# RUN FROM FROM HERE IF DATASETS NEED TO BE RESTORED
train_labels, train_images, test_images = restore()

In [6]:
# reshape labels to 1d array, encode labels to integers
train_labels = np.array(train_labels).reshape(-1)
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_y = le.transform(train_labels)

In [7]:
# check labels
print(train_labels)
print(train_y)

['shovel' 'rifle' 'scorpion' ..., 'rollerskates' 'mouth' 'pencil']
[25 20 23 ..., 21  3 13]


In [70]:
print(train_images[3])

[3 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]


In [59]:
print(train_images[0].shape)
print(train_images.shape)
print(train_images)

(2,)
(10000, 2)
[[0 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
 [1 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
 [2 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
 ..., 
 [9997 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
 [9998 array([ 255.,  255.,  255., ...,    0.,    0.,    0.])]
 [9999 array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]]


In [60]:
# reshape images
train_images = train_images.reshape(-1, 10000)
train_images = train_images.reshape(-1, 100, 100)

In [61]:
#print(train_images[0].shape)
print(train_images.shape)
print(train_images)

(2, 100, 100)
[[[0 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 1 ...,
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 49
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
  [50 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 51 ...,
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 99
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
  [100 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 101 ...,
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 149
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
  ..., 
  [4850 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 4851 ...,
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 4899
   array([ 255.,   73.,   23., ...,    0.,    0.,    0.])]
  [4900 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 4901 ...,
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 4949
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.])]
  [4950 array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 4951 ...,
   array([ 0.,  0.,  0., ...,  0.,  0.,  0.]) 4999
   array([ 14.,   0.,   0., ...,   0.,   0.,   0.])]]

 [[5000 array([ 0.,  

In [42]:
print(train_images[0][2][0])

100


In [30]:
plt.imshow(train_images[0], cmap='gray_r')

TypeError: Image data can not convert to float